In [11]:
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import folium
from folium.plugins import MarkerCluster
import re

In [12]:
url = 'https://www.mcdonalds.co.id/location'

In [15]:
# Set up Selenium WebDriver (you may need to specify the path to your webdriver executable)
driver = webdriver.Chrome()

# Load the webpage
driver.get(url)

# Wait for the page to load (you may need to adjust the wait time)
driver.implicitly_wait(5)

# Get the page source after the dynamic content has loaded
page_source = driver.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the div with id "listings"
listings_div = soup.find('div', {'id': 'listings'})

# Create an empty list to store data
data = []

# Iterate over each location-list div
for location_div in listings_div.find_all('div', class_='location-list'):
    title = location_div.find('h5').text
    address = location_div.find('p', class_='quiet').text
    directions_link = location_div.find('a', class_='btn-directions')['href']
    telephone_link = location_div.find('a', class_='btn-telephone')['href']
    
    # Extract latitude and longitude from the Directions link
    coordinates_match = re.search(r'(-?\d+\.\d+),(-?\d+\.\d+)', directions_link)
    if coordinates_match:
        latitude, longitude = coordinates_match.group(1), coordinates_match.group(2)
    else:
        latitude, longitude = None, None
    
    # Append data to the list
    data.append({
        'Title': title,
        'Address': address,
        'Directions': directions_link,
        'Telephone': telephone_link,
        'Latitude': latitude,
        'Longitude': longitude
    })

# Close the Selenium WebDriver
driver.quit()

# Create a DataFrame from the list of data
df = pd.DataFrame(data)

In [16]:
df

,Title,Address,Directions,Telephone,Latitude,Longitude
0,McDonald's Hero Bekasi,"Jl. K.H. Noer Ali No. 177 Bekasi Selatan, Beka...",https://www.google.com/maps/dir/Current+Locati...,tel:14045,-6.24635,106.991817
1,McDonald's Sarinah Malang,"Ground Floor,Jl.Jend.Basuki Rachmat 2A",https://www.google.com/maps/dir/Current+Locati...,tel:14045,-7.981156,112.630642
2,McDonald's Malioboro Jogja,"Ground Floor, Jl. Malioboro 52-58",https://www.google.com/maps/dir/Current+Locati...,tel:14045,-7.7984,110.365964
3,McDonald's Citraland Semarang,"Ground Floor, Jl. Simpang Lima Raya No. 1",https://www.google.com/maps/dir/Current+Locati...,tel:14045,-6.989375,110.424192
4,McDonald's Tebet Shophouse,"Jl. Prof. Dr. Soepomo SH, No. 55, Tebet",https://www.google.com/maps/dir/Current+Locati...,tel:14045,-6.2321,106.845578
...,...,...,...,...,...,...
304,McDonald's Pekalongan,Jl. Urip Sumoharjo No.58 Kelurahan Podosugih K...,https://www.google.com/maps/dir/Current+Locati...,tel:14045,024.965,1.096
305,McDonald's Subang,Jl. Otto Iskandardinata No 70,https://www.google.com/maps/dir/Current+Locati...,tel:14045,-6.563642,107.766987
306,McDonald's Ternate,"Jl. Sultan M. Jhabir Sjah, Gamalama Kota Ternate",https://www.google.com/maps/dir/Current+Locati...,tel:14045,0.7928532110053871,127.39004468221123
307,McDonald's Tarakan,Jl. Yos Sudarso. Kelurahan Karang Balik Kecama...,https://www.google.com/maps/dir/Current+Locati...,tel:14045,3.305301,117.584761


### Check For Non Valid Location

In [22]:
# Drop rows with None values in Latitude and Longitude columns
df = df.dropna(subset=['Latitude', 'Longitude'])

### Visualization All Location

In [23]:
# Create a Folium map centered around Jakarta
map_center = [-6.2088, 106.8456]  # Jakarta's latitude and longitude
mcd_map = folium.Map(location=map_center, zoom_start=10)

# Create a MarkerCluster layer
marker_cluster = MarkerCluster().add_to(mcd_map)

# Add markers for each location to the MarkerCluster layer
for index, row in df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['Title']).add_to(marker_cluster)

In [24]:
mcd_map